In [ ]:
#!pip install -qU transformers accelerate bitsandbytes
#!huggingface-cli download dfurman/Qwen2-72B-Orpo-v0.1

In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig
import transformers
import torch

if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

model = "dfurman/Llama-3-70B-Orpo-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch_dtype,
        # "quantization_config": bnb_config,
        "device_map": "auto",
        "attn_implementation": attn_implementation,
    },
)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me a recipe for a spicy margarita."},
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print("***Prompt:\n", prompt)

outputs = pipeline(
    prompt, max_new_tokens=1000, do_sample=True, temperature=0.7, top_k=50, top_p=0.95
)
print("***Generation:\n", outputs[0]["generated_text"][len(prompt) :])